In [1]:
import math

def pipeline_sizing(mass_flow_rate, density, viscosity, pipe_length, max_pressure_drop, max_velocity):
    """
    Perform pipeline sizing based on engineering procedures.

    Parameters:
        mass_flow_rate (float): Mass flow rate (kg/s).
        density (float): Fluid density (kg/m^3).
        viscosity (float): Fluid dynamic viscosity (Pa.s).
        pipe_length (float): Length of the pipeline (m).
        max_pressure_drop (float): Maximum allowable pressure drop (Pa).
        max_velocity (float): Maximum allowable velocity (m/s).

    Returns:
        dict: Contains the calculated parameters (pipe diameter, velocity, Reynolds number, friction factor, pressure drop).
    """

    def reynolds_number(diameter, velocity):
        return (density * velocity * diameter) / viscosity

    def darcy_weisbach_friction_factor(re):
        if re < 2000:
            # Laminar flow
            return 64 / re
        else:
            # Turbulent flow (approximation using Blasius equation)
            return 0.3164 / (re ** 0.25)

    def pressure_drop(friction_factor, velocity, diameter):
        return friction_factor * (pipe_length / diameter) * (density * velocity**2 / 2)

    # Start sizing
    diameter = 0.05  # Initial guess for diameter in meters
    velocity = 0
    re = 0
    friction_factor = 0
    dp = 0

    while True:
        # Calculate velocity from the continuity equation
        area = math.pi * (diameter / 2) ** 2
        velocity = mass_flow_rate / (density * area)

        if velocity > max_velocity:
            # Increase diameter if velocity exceeds the maximum
            diameter += 0.01
            continue

        # Calculate Reynolds number
        re = reynolds_number(diameter, velocity)

        # Calculate friction factor
        friction_factor = darcy_weisbach_friction_factor(re)

        # Calculate pressure drop
        dp = pressure_drop(friction_factor, velocity, diameter)

        if dp > max_pressure_drop:
            # Increase diameter if pressure drop exceeds the maximum
            diameter += 0.01
            continue

        # Check if all conditions are satisfied
        if velocity <= max_velocity and dp <= max_pressure_drop:
            break
    
    return {
        "Pipe Diameter (m)": diameter,
        "Pipe Diameter Calculated (inch)" : diameter*100/2.54,
        "Velocity (m/s)": velocity,
        "Reynolds Number": re,
        "Friction Factor": friction_factor,
        "Pressure Drop (Pa)": dp,
        "Pipe Diameter (inch)" : diameter*100/2.54,
    }



In [5]:
result = pipeline_sizing(
    mass_flow_rate=22.09583333333333,  # kg/s
    density=645.8,      # kg/m^3
    viscosity=0.001,     # Pa.s
    pipe_length=100,    # m
    max_pressure_drop= 50000.0,  # Pa
    max_velocity= 3.0     # m/s
)

for key, value in result.items():
    print(f"{key}: {value}")


Pipe Diameter (m): 0.12999999999999998
Pipe Diameter Calculated (inch): 5.118110236220471
Velocity (m/s): 2.577720104404919
Reynolds Number: 216409.9136452105
Friction Factor: 0.01466956334800766
Pressure Drop (Pa): 24211.03731670824
Pipe Diameter (inch): 5.118110236220471


# Pipeline_Thickness


In [1]:
import math

def barlow_thickness(P, D, S, E=1.0):
    """
    Calculate pipe wall thickness using Barlow's Formula.
    P: Internal pressure (psi or Pa)
    D: Outside diameter (in or mm)
    S: Allowable stress (psi or Pa)
    E: Weld joint efficiency (default=1.0 for seamless pipes)
    """
    t = (P * D) / (2 * S * E)
    return t

def asme_b31_3_thickness(P, D, S, Y, E=1.0):
    """
    Calculate pipe wall thickness using ASME B31.3 formula.
    P: Internal pressure (psi or Pa)
    D: Outside diameter (in or mm)
    S: Allowable stress (psi or Pa)
    Y: Wall thickness coefficient (usually 0.4 for typical steel pipes)
    E: Weld joint efficiency (default=1.0 for seamless pipes)
    """
    t = (P * D) / (2 * (S * E + P * Y))
    return t

if __name__ == "__main__":
    # User inputs
    P = float(input("Enter internal pressure (psi or Pa): "))
    D = float(input("Enter pipe outside diameter (in or mm): "))
    S = float(input("Enter allowable stress (psi or Pa): "))
    Y = float(input("Enter wall thickness coefficient (Y) [ASME B31.3]: "))
    E = float(input("Enter weld joint efficiency (default=1.0 for seamless pipes): ") or 1.0)

    # Calculate thickness using both formulas
    t_barlow = barlow_thickness(P, D, S, E)
    t_asme = asme_b31_3_thickness(P, D, S, Y, E)

    # Print results
    print("\nCalculated Pipe Thickness:")
    print(f"Barlow's Formula: {t_barlow:.4f} units")
    print(f"ASME B31.3: {t_asme:.4f} units")



Calculated Pipe Thickness:
Barlow's Formula: 36.6667 units
ASME B31.3: 9.3220 units
